In [15]:
import datasets
from datasets import load_dataset
#import numpy==1.24 as np
import pandas as pd
import seaborn as sns
import transformers
from transformers import BertTokenizer, BertModel
import torch
from sklearn.metrics.pairwise import cosine_similarity
from nltk.util import ngrams
from collections import Counter
import imodelsx
import collections

!pip install numpy==1.24  # Replace '1.24' with the specific version you want



## Load DataSet

In [16]:
ds = load_dataset("sst2")

In [92]:
# dataset description


In [93]:
#ds.info.features

## Sample very first two reviews from train DS

In [21]:
# Extract the first two reviews
rev1 = ds["train"]["sentence"][0]
rev2 = ds["train"]["sentence"][1]

# Print the reviews
print("First Review:", rev1)
print("Second Review:", rev2)


First Review: hide new secretions from the parental units 
Second Review: contains no wit , only labored gags 


## Compute Embedding w/ simple ```bert-uncased```

In [10]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Tokenize and encode the reviews
tokenized_reviews = tokenizer([rev1, rev2], padding=True, truncation=True, return_tensors="pt")

# Get the embeddings
with torch.no_grad():
    outputs = model(**tokenized_reviews)

# Extract the embeddings for each review
review1_embedding = outputs.last_hidden_state[0][0]
review2_embedding = outputs.last_hidden_state[1][0]

#print(review1_embedding)

In [22]:
from transformers import BertTokenizer, BertModel
import torch

# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Define your sample sentences
sentences = [rev1, rev2]

# Tokenize and encode the sentences
tokenized_sentences = tokenizer(sentences, padding=True, truncation=True, return_tensors="pt")

# Get the embeddings
with torch.no_grad():
    # Pass the tokenized input through the model
    outputs = model(**tokenized_sentences)

# Extract the embeddings for each sentence
# Here, we assume that the number of sentences is the same as the number of rows in the input
# and that you want to extract the embeddings for each sentence separately
embeddings = outputs.last_hidden_state

# Now, you have the embeddings for each sentence in the `embeddings` tensor
# You can access them like this:
sentence1_embedding = embeddings[0]
sentence2_embedding = embeddings[1]

# Ensure the embeddings have the same shape
print("Sentence 1 Embedding Shape:", sentence1_embedding.shape)
print("Sentence 2 Embedding Shape:", sentence2_embedding.shape)


Sentence 1 Embedding Shape: torch.Size([11, 768])
Sentence 2 Embedding Shape: torch.Size([11, 768])


## Compute similarity w/ Cosine

In [23]:
# Calculate cosine similarity
similarity_score = cosine_similarity(review1_embedding.reshape(1, -1), review2_embedding.reshape(1, -1))[0][0]

print("Cosine Similarity:", similarity_score)

Cosine Similarity: 0.69258183


## Feature importance

In [24]:
# Function to extract n-grams from a sentence
def extract_ngrams(sentence, n):
    tokens = sentence.split()
    ngrams_list = list(ngrams(tokens, n))
    return [' '.join(gram) for gram in ngrams_list]

# Function to calculate n-gram importance
def calculate_ngram_importance(rev1, rev2, n):
    # Extract n-grams from both reviews
    ngrams_review1 = extract_ngrams(rev1, n)
    ngrams_review2 = extract_ngrams(rev2, n)
    
    # Count the frequency of each n-gram in both reviews
    ngram_counts_review1 = Counter(ngrams_review1)
    ngram_counts_review2 = Counter(ngrams_review2)
    
    # Calculate the intersection of n-grams
    intersection_ngrams = set(ngram_counts_review1.keys()) & set(ngram_counts_review2.keys())
    
    # Calculate the importance of each common n-gram as the sum of their frequencies
    ngram_importance = {ngram: ngram_counts_review1[ngram] + ngram_counts_review2[ngram] for ngram in intersection_ngrams}
    
    # Sort n-grams by importance (descending)
    sorted_ngram_importance = dict(sorted(ngram_importance.items(), key=lambda item: item[1], reverse=True))
    
    return sorted_ngram_importance


## Compare with lime/ Shap

In [25]:
import shap

# Define a function that computes similarity using cosine similarity
def predict_similarity(texts):
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    #embeddings = [model(**tokenizer(text, padding=True, truncation=True, return_tensors="pt").to(device)).last_hidden_state[0][0] for text in texts]
    similarity = cosine_similarity(review1_embedding[0].reshape(1, -1), review2_embedding[1].reshape(1, -1))[0][0]
    return [1 - similarity, similarity]

# Create a SHAP explainer
explainer = shap.Explainer(predict_similarity, tokenizer)

# Explain the model's predictions
shap_values = explainer([rev1, rev2])

# Visualize SHAP explanation
shap.summary_plot(shap_values, tokenizer.vocab, show=False)



AssertionError: The model produced 2 output rows when given 1 input rows! Check the implementation of the model you provided for errors.